In [ ]:
# 11_build_features_offline.ipynb

import numpy as np
import pandas as pd
from pathlib import Path
from decimal import Decimal, InvalidOperation
from collections import defaultdict, deque

import networkx as nx
from node2vec import Node2Vec

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 200)

# ---------------------------
# Пути проекта
# ---------------------------

PROJECT_ROOT = Path(".")
DATA_ROOT = PROJECT_ROOT / "data"
RAW_PATH = DATA_ROOT / "raw"
PROC_PATH = DATA_ROOT / "processed"
MODELS_PATH = PROJECT_ROOT / "models"

RAW_PATH.mkdir(parents=True, exist_ok=True)
PROC_PATH.mkdir(parents=True, exist_ok=True)
MODELS_PATH.mkdir(parents=True, exist_ok=True)

RAW_TX_CSV = RAW_PATH / "транзакции_в_Мобильном_интернет_Банкинге.csv"
RAW_BEH_CSV = RAW_PATH / "поведенческие_паттерны_клиентов_3.csv"

CLEAN_TX_PARQUET = PROC_PATH / "transactions_clean.parquet"
FEATURES_PARQUET = PROC_PATH / "features_offline_v11.parquet"

CST_EMB_PARQUET = MODELS_PATH / "node2vec_cst_dim_id.parquet"
DIR_EMB_PARQUET = MODELS_PATH / "node2vec_direction.parquet"


# ============================================================
# 1. Загрузка и очистка данных
# ============================================================

def load_and_clean_transactions(raw_csv_path: Path) -> pd.DataFrame:
    """
    Загружает транзакции, чинит два заголовка и типы.
    Выходная схема:
      cst_dim_id (str)
      transdate (date)
      transdatetime (Timestamp)
      amount (float)
      docno (int)
      direction (str)
      target (int)
      row_id (int, по времени)
    """
    df_raw = pd.read_csv(raw_csv_path, encoding="cp1251", sep=";")

    # первая строка — машинные имена колонок
    machine_cols = list(df_raw.iloc[0])
    df = df_raw.iloc[1:].copy()
    df.columns = machine_cols

    # даты/время
    df["transdatetime"] = (
        df["transdatetime"].astype(str).str.strip().str.strip("'")
    )
    df["transdate"] = (
        df["transdate"].astype(str).str.strip().str.strip("'")
    )

    df["transdatetime"] = pd.to_datetime(df["transdatetime"], errors="coerce")
    df["transdate"] = pd.to_datetime(df["transdate"], errors="coerce").dt.date

    # суммы и идентификаторы
    df["amount"] = pd.to_numeric(df["amount"], errors="coerce")
    df["docno"] = pd.to_numeric(df["docno"], errors="coerce")
    df["target"] = (
        pd.to_numeric(df["target"], errors="coerce")
        .fillna(0)
        .astype(int)
    )

    df["cst_dim_id"] = df["cst_dim_id"].astype(str).str.strip()
    df["direction"] = df["direction"].astype(str).str.strip()

    # выкидываем критично битые строки
    df = df.dropna(subset=["transdatetime", "transdate", "amount", "docno"])
    df["docno"] = df["docno"].astype(int)

    # сортируем по времени и даём стабильный row_id
    df = df.sort_values(["transdatetime", "docno"]).reset_index(drop=True)
    df["row_id"] = np.arange(len(df), dtype=np.int64)

    return df


def load_and_clean_behaviour(raw_csv_path: Path) -> pd.DataFrame:
    """
    Загружает датасет поведенческих паттернов клиентов (логины и сессии),
    чинит заголовки, типы, переименовывает колонки в sess_*
    и аггрегирует до одного ряда на (cst_dim_id, transdate).

    Ключ для join: (cst_dim_id, transdate)
    """
    df_raw = pd.read_csv(raw_csv_path, encoding="cp1251", sep=";")

    # первая строка — машинные имена колонок
    machine_cols = list(df_raw.iloc[0])
    df = df_raw.iloc[1:].copy()
    df.columns = machine_cols

    # ключи
    df["transdate"] = (
        df["transdate"].astype(str).str.strip().str.strip("'")
    )
    df["transdate"] = pd.to_datetime(df["transdate"], errors="coerce").dt.date
    df["cst_dim_id"] = df["cst_dim_id"].astype(str).str.strip()

    # аккуратное переименование полей → sess_*
    rename_map = {
        "monthly_os_changes": "sess_monthly_os_changes",
        "monthly_phone_model_changes": "sess_monthly_phone_model_changes",
        "last_phone_model_categorical": "sess_last_phone_model",
        "last_os_categorical": "sess_last_os",
        "logins_last_7_days": "sess_logins_7d",
        "logins_last_30_days": "sess_logins_30d",
        "login_frequency_7d": "sess_login_freq_7d",
        "login_frequency_30d": "sess_login_freq_30d",
        "freq_change_7d_vs_mean": "sess_freq_change_7d_vs_mean",
        "logins_7d_over_30d_ratio": "sess_logins_7d_30d_ratio",
        "avg_login_interval_30d": "sess_avg_login_interval_30d",
        "std_login_interval_30d": "sess_std_login_interval_30d",
        "var_login_interval_30d": "sess_var_login_interval_30d",
        "ewm_login_interval_7d": "sess_ewm_login_interval_7d",
        "burstiness_login_interval": "sess_burstiness_login_interval",
        "fano_factor_login_interval": "sess_fano_login_interval",
        "zscore_avg_login_interval_7d": "sess_z_login_interval_7d",
    }
    df = df.rename(columns=rename_map)

    sess_cat_cols = ["sess_last_phone_model", "sess_last_os"]

    # числовые колонки
    for col in df.columns:
        if col in ["transdate", "cst_dim_id"] + sess_cat_cols:
            continue
        df[col] = pd.to_numeric(df[col], errors="coerce")

    # sentinel -1 → NaN для интервальных метрик (нет истории)
    sentinel_cols = [
        "sess_avg_login_interval_30d",
        "sess_std_login_interval_30d",
        "sess_var_login_interval_30d",
        "sess_ewm_login_interval_7d",
        "sess_burstiness_login_interval",
        "sess_fano_login_interval",
        "sess_z_login_interval_7d",
    ]
    for col in sentinel_cols:
        if col in df.columns:
            df[col] = df[col].replace(-1, np.nan)

    # флаг "вообще есть история логинов" на этом горизонте
    interval_cols_existing = [c for c in sentinel_cols if c in df.columns]
    if interval_cols_existing:
        df["sess_has_login_history"] = (
            ~df[interval_cols_existing].isna()
        ).any(axis=1).astype(int)
    else:
        df["sess_has_login_history"] = 0

    # --- Агрегация до одного ряда на (cst_dim_id, transdate) ---

    key_cols = ["cst_dim_id", "transdate"]

    # числовые колонки
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    # sess_has_login_history агрегируем отдельно
    num_cols_wo_hist = [c for c in num_cols if c != "sess_has_login_history"]

    agg_dict = {col: "mean" for col in num_cols_wo_hist}
    agg_dict["sess_has_login_history"] = "max"

    for col in sess_cat_cols:
        if col in df.columns:
            # последняя категория за день (по исходному порядку)
            agg_dict[col] = "last"

    df_grouped = (
        df.groupby(key_cols, as_index=False)
          .agg(agg_dict)
    )

    # числовые NaN → 0
    num_cols_grp = df_grouped.select_dtypes(include=[np.number]).columns
    df_grouped[num_cols_grp] = df_grouped[num_cols_grp].fillna(0.0)

    # категориальные NaN → "unknown"
    for col in sess_cat_cols:
        if col in df_grouped.columns:
            df_grouped[col] = df_grouped[col].fillna("unknown").astype(str)

    df_grouped = df_grouped.dropna(subset=["transdate", "cst_dim_id"]).reset_index(drop=True)

    # гарантируем уникальность ключей (для validate='m:1')
    assert not df_grouped.duplicated(subset=["cst_dim_id", "transdate"]).any(), \
        "Все равно есть дубликаты по (cst_dim_id, transdate) в поведении"

    return df_grouped


# ============================================================
# 2. Базовые фичи (время, decimal_depth, лог-суммы)
# ============================================================

def decimal_depth_amount(x: float) -> int:
    """
    Количество значащих знаков после запятой у числа.
    Через Decimal.normalize() — корректно для 0.100000 и т.п.
    """
    try:
        d = Decimal(str(x)).normalize()
        exp = -d.as_tuple().exponent
        return max(exp, 0)
    except (InvalidOperation, TypeError, ValueError):
        return 0


def add_base_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Добавляет:
      hour, dayofweek, is_weekend,
      hour_sin/cos, dow_sin/cos,
      log_amount, decimal_depth.
    """
    df = df.copy()

    df["hour"] = df["transdatetime"].dt.hour.astype(int)
    df["dayofweek"] = df["transdatetime"].dt.dayofweek.astype(int)
    df["is_weekend"] = df["dayofweek"].isin([5, 6]).astype(int)

    # циклическое кодирование времени
    df["hour_sin"] = np.sin(2 * np.pi * df["hour"] / 24.0)
    df["hour_cos"] = np.cos(2 * np.pi * df["hour"] / 24.0)

    df["dow_sin"] = np.sin(2 * np.pi * df["dayofweek"] / 7.0)
    df["dow_cos"] = np.cos(2 * np.pi * df["dayofweek"] / 7.0)

    # логарифм суммы
    df["log_amount"] = np.log1p(df["amount"].clip(lower=0))

    # финансовая "биометрия" суммы
    df["decimal_depth"] = df["amount"].apply(decimal_depth_amount).astype(int)

    return df


# ============================================================
# 3. Исторические агрегаты 7/30/90 дней + Z-score
# ============================================================

def add_rolling_aggregates(df: pd.DataFrame) -> pd.DataFrame:
    """
    По каждому клиенту:
      user_mean_amount_{7d,30d,90d}
      user_std_amount_{7d,30d,90d}
      user_max_amount_{7d,30d,90d}
      user_min_amount_{7d,30d,90d}
      user_tx_count_{7d,30d,90d}
      z_amount_30d — Z-score суммы относительно 30-дневного окна.
    """
    df = df.copy()
    df = df.sort_values(["cst_dim_id", "transdatetime"]).reset_index(drop=True)

    # удобнее сделать индекс = время
    df = df.set_index("transdatetime")
    group_amt = df.groupby("cst_dim_id")["amount"]

    def add_window_stats(window_label: str, window: str):
        roll = group_amt.rolling(window, closed="left")

        df[f"user_mean_amount_{window_label}"] = (
            roll.mean().reset_index(level=0, drop=True)
        )
        df[f"user_std_amount_{window_label}"] = (
            roll.std(ddof=0).reset_index(level=0, drop=True)
        )
        df[f"user_max_amount_{window_label}"] = (
            roll.max().reset_index(level=0, drop=True)
        )
        df[f"user_min_amount_{window_label}"] = (
            roll.min().reset_index(level=0, drop=True)
        )
        df[f"user_tx_count_{window_label}"] = (
            roll.count().reset_index(level=0, drop=True)
        )

    add_window_stats("7d", "7D")
    add_window_stats("30d", "30D")
    add_window_stats("90d", "90D")

    # Z-score по 30 дням для суммы
    mean_30 = df["user_mean_amount_30d"]
    std_30 = df["user_std_amount_30d"]
    df["z_amount_30d"] = (df["amount"] - mean_30) / (std_30 + 1e-6)
    df["z_amount_30d"] = df["z_amount_30d"].fillna(0.0)

    df = df.reset_index()  # вернём transdatetime в колонки
    return df


# ============================================================
# 4. Offline velocity (клиент / получатель)
# ============================================================

def add_offline_velocity(df: pd.DataFrame) -> pd.DataFrame:
    """
    Velocity-фичи по истории до текущего события:

    Для клиента:
      user_tx_1m, user_tx_10m, user_tx_60m,
      user_sum_60m,
      user_new_dirs_60m — количество уникальных получателей за 60 мин.

    Для получателя:
      dir_tx_60m,
      dir_unique_senders_60m — уникальных клиентов за 60 мин.
    """
    df = df.copy()
    df = df.sort_values("transdatetime").reset_index(drop=True)
    n = len(df)

    user_tx_1m = np.zeros(n, dtype=np.int32)
    user_tx_10m = np.zeros(n, dtype=np.int32)
    user_tx_60m = np.zeros(n, dtype=np.int32)
    user_sum_60m = np.zeros(n, dtype=np.float32)
    user_new_dirs_60m = np.zeros(n, dtype=np.int32)

    dir_tx_60m = np.zeros(n, dtype=np.int32)
    dir_unique_senders_60m = np.zeros(n, dtype=np.int32)

    win_1m = 60
    win_10m = 600
    win_60m = 3600

    user_events = defaultdict(deque)  # cst_dim_id -> deque[(ts, amount, direction)]
    dir_events = defaultdict(deque)   # direction   -> deque[(ts, cst_dim_id, amount)]

    for i, row in df.iterrows():
        ts = row["transdatetime"].timestamp()
        cst = row["cst_dim_id"]
        direc = row["direction"]
        amt = float(row["amount"])

        # ---- клиент ----
        dq_user = user_events[cst]

        # чистим хвост старше 60 минут
        while dq_user and (ts - dq_user[0][0]) > win_60m:
            dq_user.popleft()

        if dq_user:
            times_user = [t for t, _, _ in dq_user]

            user_tx_1m[i] = sum(ts - t <= win_1m for t in times_user)
            user_tx_10m[i] = sum(ts - t <= win_10m for t in times_user)
            user_tx_60m[i] = len(times_user)

            user_sum_60m[i] = sum(
                a for t, a, _ in dq_user if ts - t <= win_60m
            )

            recent_dirs_60m = {
                d for t, _, d in dq_user if ts - t <= win_60m
            }
            user_new_dirs_60m[i] = len(recent_dirs_60m)
        else:
            user_tx_1m[i] = 0
            user_tx_10m[i] = 0
            user_tx_60m[i] = 0
            user_sum_60m[i] = 0.0
            user_new_dirs_60m[i] = 0

        # добавляем текущую транзакцию в историю клиента
        dq_user.append((ts, amt, direc))

        # ---- получатель ----
        dq_dir = dir_events[direc]
        while dq_dir and (ts - dq_dir[0][0]) > win_60m:
            dq_dir.popleft()

        if dq_dir:
            times_dir = [t for t, _, _ in dq_dir]
            senders_dir = [c for _, c, _ in dq_dir]

            dir_tx_60m[i] = len(times_dir)
            dir_unique_senders_60m[i] = len(set(senders_dir))
        else:
            dir_tx_60m[i] = 0
            dir_unique_senders_60m[i] = 0

        dq_dir.append((ts, cst, amt))

    df["user_tx_1m"] = user_tx_1m
    df["user_tx_10m"] = user_tx_10m
    df["user_tx_60m"] = user_tx_60m
    df["user_sum_60m"] = user_sum_60m
    df["user_new_dirs_60m"] = user_new_dirs_60m

    df["dir_tx_60m"] = dir_tx_60m
    df["dir_unique_senders_60m"] = dir_unique_senders_60m

    return df


# ============================================================
# 5. Онлайновые графовые фичи (degree / fraud_share)
# ============================================================

def add_graph_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Граф клиент–получатель, считаемый онлайново по истории:

      degree_cst — сколько уникальных получателей у клиента
      degree_dir — сколько уникальных клиентов у получателя

      cst_fraud_share — доля фродовых транзакций клиента в прошлом
      dir_fraud_share — доля фродовых транзакций получателя в прошлом

      one_to_many_flag — клиент с очень высокой степенью (раздающий)
      many_to_one_flag — получатель, собирающий много клиентов (mule)
    """
    df = df.copy()
    df = df.sort_values("transdatetime").reset_index(drop=True)
    n = len(df)

    degree_cst = np.zeros(n, dtype=np.int32)
    degree_dir = np.zeros(n, dtype=np.int32)
    cst_fraud_share = np.zeros(n, dtype=np.float32)
    dir_fraud_share = np.zeros(n, dtype=np.float32)

    cst_neighbors = defaultdict(set)
    dir_neighbors = defaultdict(set)

    cst_fraud_count = defaultdict(int)
    cst_tx_count = defaultdict(int)
    dir_fraud_count = defaultdict(int)
    dir_tx_count = defaultdict(int)

    for i, row in df.iterrows():
        cst = row["cst_dim_id"]
        direc = row["direction"]
        y = int(row["target"])

        # обновляем степени по уникальным соседям
        cst_neighbors[cst].add(direc)
        dir_neighbors[direc].add(cst)
        degree_cst[i] = len(cst_neighbors[cst])
        degree_dir[i] = len(dir_neighbors[direc])

        # fraud_share — по истории до текущей транзакции
        cst_fraud_share[i] = (
            cst_fraud_count[cst] / cst_tx_count[cst]
            if cst_tx_count[cst] > 0 else 0.0
        )
        dir_fraud_share[i] = (
            dir_fraud_count[direc] / dir_tx_count[direc]
            if dir_tx_count[direc] > 0 else 0.0
        )

        # теперь включаем текущую транзакцию в историю
        cst_tx_count[cst] += 1
        dir_tx_count[direc] += 1
        if y == 1:
            cst_fraud_count[cst] += 1
            dir_fraud_count[direc] += 1

    df["degree_cst"] = degree_cst
    df["degree_dir"] = degree_dir
    df["cst_fraud_share"] = cst_fraud_share
    df["dir_fraud_share"] = dir_fraud_share

    # пороги для many-to-one / one-to-many
    cst_threshold = np.quantile(degree_cst, 0.90) if n > 0 else 0
    dir_threshold = 2  # можно потом заменить на квантиль

    df["one_to_many_flag"] = (df["degree_cst"] >= cst_threshold).astype(int)
    df["many_to_one_flag"] = (df["degree_dir"] >= dir_threshold).astype(int)

    return df


# ============================================================
# 6. Node2Vec (глобовый граф) + merge в фичи
# ============================================================

def compute_node2vec_embeddings(
    df: pd.DataFrame,
    emb_dim: int = 64,
    walk_length: int = 30,
    num_walks: int = 20,
    window: int = 10,
) -> (pd.DataFrame, pd.DataFrame):
    """
    Двудольный граф C_{cst_dim_id} — D_{direction}.

    Вес ребра:
      (log1p(amount) + 1) * exp(-age_days / 30)

    Возвращает:
      cst_emb: cst_dim_id, emb_cst_0..emb_cst_{emb_dim-1}
      dir_emb: direction, emb_dir_0..emb_dir_{emb_dim-1}
    """
    edges = df[["cst_dim_id", "direction", "amount", "transdatetime"]].copy()

    max_time = edges["transdatetime"].max()
    age_days = (max_time - edges["transdatetime"]).dt.total_seconds() / (3600 * 24)

    # экспоненциальное затухание по давности
    decay = np.exp(-age_days / 30.0)
    edges["weight"] = (np.log1p(edges["amount"].clip(lower=0)) + 1.0) * decay

    G = nx.Graph()
    for _, row in edges.iterrows():
        c_node = f"C_{row['cst_dim_id']}"
        d_node = f"D_{row['direction']}"
        w = float(row["weight"])

        if G.has_edge(c_node, d_node):
            G[c_node][d_node]["weight"] += w
        else:
            G.add_edge(c_node, d_node, weight=w)

    if G.number_of_nodes() == 0:
        cst_emb = pd.DataFrame(columns=["cst_dim_id"])
        dir_emb = pd.DataFrame(columns=["direction"])
        return cst_emb, dir_emb

    n2v = Node2Vec(
        G,
        dimensions=emb_dim,
        walk_length=walk_length,
        num_walks=num_walks,
        weight_key="weight",
        workers=1,
        quiet=True,
    )
    model = n2v.fit(window=window, min_count=1, batch_words=128)

    nodes = list(model.wv.index_to_key)
    vectors = model.wv.vectors

    emb_df = pd.DataFrame(vectors, columns=[f"emb_{i}" for i in range(emb_dim)])
    emb_df["node"] = nodes
    emb_df["kind"] = emb_df["node"].str[0]   # 'C' или 'D'
    emb_df["id"] = emb_df["node"].str[2:]    # убираем 'C_' / 'D_'

    cst_emb = emb_df[emb_df["kind"] == "C"].copy()
    dir_emb = emb_df[emb_df["kind"] == "D"].copy()

    cst_emb = cst_emb.drop(columns=["kind", "node"]).rename(columns={"id": "cst_dim_id"})
    dir_emb = dir_emb.drop(columns=["kind", "node"]).rename(columns={"id": "direction"})

    cst_emb["cst_dim_id"] = cst_emb["cst_dim_id"].astype(str)
    dir_emb["direction"] = dir_emb["direction"].astype(str)

    cst_emb = cst_emb.rename(columns={f"emb_{i}": f"emb_cst_{i}" for i in range(emb_dim)})
    dir_emb = dir_emb.rename(columns={f"emb_{i}": f"emb_dir_{i}" for i in range(emb_dim)})

    return cst_emb, dir_emb


def add_node2vec_to_features(
    df: pd.DataFrame,
    cst_emb: pd.DataFrame,
    dir_emb: pd.DataFrame,
) -> pd.DataFrame:
    """
    Джоинит эмбеддинги по cst_dim_id и direction.
    Пустые эмбеддинги → 0.
    """
    df = df.copy()
    df["cst_dim_id"] = df["cst_dim_id"].astype(str)
    df["direction"] = df["direction"].astype(str)

    if not cst_emb.empty:
        df = df.merge(cst_emb, on="cst_dim_id", how="left")
    if not dir_emb.empty:
        df = df.merge(dir_emb, on="direction", how="left")

    emb_cols = [c for c in df.columns if c.startswith("emb_cst_") or c.startswith("emb_dir_")]
    if emb_cols:
        df[emb_cols] = df[emb_cols].fillna(0.0)

    return df


# ============================================================
# 7. Главная функция сборки Feature Store
# ============================================================

def build_features_offline() -> pd.DataFrame:
    print("1) Загрузка и очистка транзакций...")
    df_tx = load_and_clean_transactions(RAW_TX_CSV)
    df_tx.to_parquet(CLEAN_TX_PARQUET, index=False)
    print(f"   → сохранено в {CLEAN_TX_PARQUET}")

    print("2) Загрузка и агрегирование поведенческих паттернов (sess_*)...")
    if RAW_BEH_CSV.exists():
        df_beh = load_and_clean_behaviour(RAW_BEH_CSV)
        print(f"   → после агрегации: {len(df_beh):,} строк поведенческих данных")

        df = df_tx.merge(
            df_beh,
            on=["cst_dim_id", "transdate"],
            how="left",
            validate="m:1",  # теперь гарантированно один ряд на (клиент, дата)
        )

        # для клиентов/дат без поведения — нейтральные значения
        sess_num_cols = [
            c for c in df.select_dtypes(include=[np.number]).columns
            if c.startswith("sess_")
        ]
        if sess_num_cols:
            df[sess_num_cols] = df[sess_num_cols].fillna(0.0)

        sess_cat_cols = [c for c in df.columns if c.startswith("sess_last_")]
        for col in sess_cat_cols:
            df[col] = df[col].fillna("unknown").astype(str)

        # если где-то sess_has_login_history NaN — считаем, что истории нет
        if "sess_has_login_history" in df.columns:
            df["sess_has_login_history"] = df["sess_has_login_history"].fillna(0).astype(int)
    else:
        print("   ! WARNING: файл с поведенческими паттернами не найден, продолжаем без sess_* фичей")
        df = df_tx.copy()

    print("3) Базовые временные и amount-фичи...")
    df = add_base_features(df)

    print("4) Исторические агрегаты 7/30/90 дней и Z-score по сумме...")
    df = add_rolling_aggregates(df)

    print("5) Offline velocity по клиенту и получателю...")
    df = add_offline_velocity(df)

    print("6) Графовые фичи (degree, fraud_share, many-to-one / one-to-many)...")
    df = add_graph_features(df)

    print("7) Node2Vec-эмбеддинги (глобовый граф, может занять время)...")
    cst_emb, dir_emb = compute_node2vec_embeddings(df, emb_dim=64)
    cst_emb.to_parquet(CST_EMB_PARQUET, index=False)
    dir_emb.to_parquet(DIR_EMB_PARQUET, index=False)
    print(f"   → эмбеддинги сохранены в {CST_EMB_PARQUET} и {DIR_EMB_PARQUET}")

    df = add_node2vec_to_features(df, cst_emb, dir_emb)

    print("8) Сохранение итогового Offline Feature Store...")
    df.to_parquet(FEATURES_PARQUET, index=False)
    print(f"   → сохранено в {FEATURES_PARQUET}")

    return df


# Запуск пайплайна прямо из ноутбука
df_features_offline = build_features_offline()
df_features_offline.head()


1) Загрузка и очистка транзакций...
   → сохранено в data/processed/transactions_clean.parquet
2) Загрузка и агрегирование поведенческих паттернов (sess_*)...
   → после агрегации: 8,579 строк поведенческих данных
3) Базовые временные и amount-фичи...
4) Исторические агрегаты 7/30/90 дней и Z-score по сумме...
5) Offline velocity по клиенту и получателю...
6) Графовые фичи (degree, fraud_share, many-to-one / one-to-many)...
7) Node2Vec-эмбеддинги (глобовый граф, может занять время)...
   → эмбеддинги сохранены в models/node2vec_cst_dim_id.parquet и models/node2vec_direction.parquet
8) Сохранение итогового Offline Feature Store...
   → сохранено в data/processed/features_offline_v11.parquet


,transdatetime,cst_dim_id,transdate,amount,docno,direction,target,row_id,sess_monthly_os_changes,sess_monthly_phone_model_changes,sess_logins_7d,sess_logins_30d,sess_login_freq_7d,sess_login_freq_30d,sess_freq_change_7d_vs_mean,sess_logins_7d_30d_ratio,sess_avg_login_interval_30d,sess_std_login_interval_30d,sess_var_login_interval_30d,sess_ewm_login_interval_7d,sess_burstiness_login_interval,sess_fano_login_interval,sess_z_login_interval_7d,sess_has_login_history,sess_last_phone_model,sess_last_os,hour,dayofweek,is_weekend,hour_sin,hour_cos,dow_sin,dow_cos,log_amount,decimal_depth,user_mean_amount_7d,user_std_amount_7d,user_max_amount_7d,user_min_amount_7d,user_tx_count_7d,user_mean_amount_30d,user_std_amount_30d,user_max_amount_30d,user_min_amount_30d,user_tx_count_30d,user_mean_amount_90d,user_std_amount_90d,user_max_amount_90d,user_min_amount_90d,user_tx_count_90d,z_amount_30d,user_tx_1m,user_tx_10m,user_tx_60m,user_sum_60m,user_new_dirs_60m,dir_tx_60m,dir_unique_senders_60m,degree_cst,degree_dir,cst_fraud_share,dir_fraud_share,one_to_many_flag,many_to_one_flag,emb_cst_0,emb_cst_1,emb_cst_2,emb_cst_3,emb_cst_4,emb_cst_5,emb_cst_6,emb_cst_7,emb_cst_8,emb_cst_9,emb_cst_10,emb_cst_11,emb_cst_12,emb_cst_13,emb_cst_14,emb_cst_15,emb_cst_16,emb_cst_17,emb_cst_18,emb_cst_19,emb_cst_20,emb_cst_21,emb_cst_22,emb_cst_23,emb_cst_24,emb_cst_25,emb_cst_26,emb_cst_27,emb_cst_28,emb_cst_29,emb_cst_30,emb_cst_31,emb_cst_32,emb_cst_33,emb_cst_34,emb_cst_35,emb_cst_36,emb_cst_37,emb_cst_38,emb_cst_39,emb_cst_40,emb_cst_41,emb_cst_42,emb_cst_43,emb_cst_44,emb_cst_45,emb_cst_46,emb_cst_47,emb_cst_48,emb_cst_49,emb_cst_50,emb_cst_51,emb_cst_52,emb_cst_53,emb_cst_54,emb_cst_55,emb_cst_56,emb_cst_57,emb_cst_58,emb_cst_59,emb_cst_60,emb_cst_61,emb_cst_62,emb_cst_63,emb_dir_0,emb_dir_1,emb_dir_2,emb_dir_3,emb_dir_4,emb_dir_5,emb_dir_6,emb_dir_7,emb_dir_8,emb_dir_9,emb_dir_10,emb_dir_11,emb_dir_12,emb_dir_13,emb_dir_14,emb_dir_15,emb_dir_16,emb_dir_17,emb_dir_18,emb_dir_19,emb_dir_20,emb_dir_21,emb_dir_22,emb_dir_23,emb_dir_24,emb_dir_25,emb_dir_26,emb_dir_27,emb_dir_28,emb_dir_29,emb_dir_30,emb_dir_31,emb_dir_32,emb_dir_33,emb_dir_34,emb_dir_35,emb_dir_36,emb_dir_37,emb_dir_38,emb_dir_39,emb_dir_40,emb_dir_41,emb_dir_42,emb_dir_43,emb_dir_44,emb_dir_45,emb_dir_46,emb_dir_47,emb_dir_48,emb_dir_49,emb_dir_50,emb_dir_51,emb_dir_52,emb_dir_53,emb_dir_54,emb_dir_55,emb_dir_56,emb_dir_57,emb_dir_58,emb_dir_59,emb_dir_60,emb_dir_61,emb_dir_62,emb_dir_63
0,2024-11-30 05:45:05,2095314234,2024-11-30,20000.0,1560,5eb09003e1a1831ed3bf03315edaae0e,0,0,2.0,2.0,6.0,56.0,0.857143,1.866667,-0.540816,0.107143,43899.375000,69887.137765,4.884212e+09,47065.871869,0.228391,111259.261094,0.190030,1,"iPhone16,1",iOS/17.3.1,5,5,1,0.965926,0.258819,-0.974928,-0.222521,9.903538,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,0,0,0.0,0,0,0,1,1,0.0,0.0,0,0,-0.989427,0.081997,1.611601,0.951304,0.259954,0.260227,0.262570,-0.040021,-1.354929,1.053223,-0.315046,-0.549241,-0.007243,-0.053268,-0.841802,-1.313949,-0.798776,0.475227,-0.123883,0.420621,1.271067,0.635403,-0.558220,-0.976648,-0.761991,1.211800,-0.698219,-0.259085,0.358517,-0.157114,0.585319,0.063239,0.491738,0.245856,-1.456009,0.481050,0.833333,-0.670734,-0.264346,0.750464,0.816050,0.361441,-0.139248,-0.992738,-0.595023,-0.078383,-0.163794,-0.673993,-0.705029,-0.141700,0.722438,0.282234,-1.229176,-0.413882,-0.076234,-0.547116,-0.550251,-0.425416,-1.180788,-0.361189,0.305453,1.134467,-0.448969,0.276543,-0.939995,0.092112,1.639376,0.926411,0.235249,0.239935,0.250497,-0.040229,-1.338480,1.085050,-0.269612,-0.552225,-0.014821,-0.072861,-0.866540,-1.301161,-0.770786,0.496902,-0.132210,0.482559,1.222642,0.618818,-0.553844,-0.995719,-0.734860,1.182436,-0.691188,-0.258356,0.392442,-0.149309,0.606677,0.080449,0.475192,0.214289,-1.469054,0.470929,0.843846,-0.671087,-0.260576,0.758512,0.765717,0.392065,-0.117517,-0.984928,-0.621327,-0.090417,-0.192909,-0.695549,-0.670318,-0.098602,0.729526,0.242218,-1.230991,-0.